#### Features aggregation dataframes

In [ ]:
import joblib
import pandas as pd
import numpy as np

In [ ]:
inst_df = joblib.load("data/home-credit-default-risk/pkl_files/inst_df")
cc_df = joblib.load("data/home-credit-default-risk/pkl_files/cc_df")
pos_df = joblib.load("data/home-credit-default-risk/pkl_files/pos_df")
bu_mg_df = joblib.load("data/home-credit-default-risk/pkl_files/bu_mg_df")
bu_bal_df = joblib.load("data/home-credit-default-risk/pkl_files/bu_bal_df")
comb_df = joblib.load("data/home-credit-default-risk/pkl_files/comb_df")


##### instalment_agg_df

In [ ]:
# inst_agg_df
def inst_agg_df():
    """
    Aggregate installment df data
    """

    prev_curr = inst_df[
        [
            "sk_id_prev",
            "sk_id_curr",
        ]
    ].groupby(["sk_id_prev", "sk_id_curr"])
    
    prev_curr_map = list(prev_curr.groups.keys())
    
    inst_group = inst_df[
        [
            "sk_id_prev",
            "sk_id_curr",
            "num_instalment_number",
            "days_instalment",
            "days_entry_payment",
            "amt_instalment",
            "amt_payment",
        ]
    ].groupby(["sk_id_prev"])
    
    group_keys = list(inst_group.groups.keys())
    
    inst_agg_df = pd.DataFrame()
    
    total_inst_count_ls = []
    fully_paid_ratio_ls = []
    delay_inst_ratio_ls = []
    delay_total_days_ls = []
    delay_days_mean_ls = []
    total_payment_ls = []
    total_pay_owe_diff_ls = []
    total_pay_owe_diff_ratio_ls = []
    
    inst_agg_df = pd.DataFrame()
    
    
    for i in group_keys[:]:
        gp = inst_group.get_group((i,)).sort_values(by="num_instalment_number")
    
        total_inst_count = len(
            gp.groupby("num_instalment_number").days_instalment.mean().tolist()
        )
    
        # calculate fully_paid_ratio of installments
        # check if only one payment is made in one instalment
        if gp.num_instalment_number.max() == len(gp.num_instalment_number):
            inst_amt = gp.amt_instalment.tolist()
            inst_pay = gp.amt_payment.tolist()
    
        # when more than one payment is made in the same instalment, create a mapping
        else:
        
            inst_pay = gp.groupby(
                "num_instalment_number").amt_payment.sum().tolist()
    
            inst_num_and_inst_amt = sorted(
                list(
                    set(
                        list(
                            gp[["num_instalment_number", "amt_instalment"]].itertuples(
                                index=False, name=None
                            )
                        )
                    )
                ),
                key=lambda tup: tup[0],
            )
            inst_amt = [j[1] for j in inst_num_and_inst_amt]
    
        # misalignment of inst_amt and inst_pay, occurs when there's different amt for the same instalment
        if len(inst_amt) != len(inst_pay):
            mis_ratio = sum(inst_pay) / sum(inst_amt)
            fully_paid_ratio = 1 if mis_ratio >= 1 else mis_ratio
    
        else:
            fully_paid_ratio = False
    
        if not bool(fully_paid_ratio):
            fully_paid_ratio = (
                sum([inst_amt[i] >= inst_pay[i] for i in range(total_inst_count)])
                / total_inst_count
            )
    
        pay_day_ls = gp.groupby(["num_instalment_number"]
                                ).days_entry_payment.max().tolist()
        should_pay_day_ls = (
            gp.groupby(["num_instalment_number"]).days_instalment.max().tolist()
        )
        delay_pay_days_ls = [
            pay_day_ls[i] > should_pay_day_ls[i] for i in range(len(pay_day_ls))
        ]
    
        delay_inst_ratio = sum(delay_pay_days_ls) / len(delay_pay_days_ls)
        delay_total_days = sum(
            [
                pay_day_ls[i] - should_pay_day_ls[i]
                for i in range(len(pay_day_ls))
                if pay_day_ls[i] > should_pay_day_ls[i]
            ]
        )
        delay_days_mean = delay_total_days / total_inst_count
        total_payment = sum(inst_pay)
        total_pay_owe_diff = sum(inst_pay) - sum(inst_amt)
        total_pay_owe_diff_ratio = total_pay_owe_diff / sum(inst_pay)
    
        total_inst_count_ls.append(total_inst_count)
        fully_paid_ratio_ls.append(fully_paid_ratio)
    
        delay_inst_ratio_ls.append(delay_inst_ratio)
        delay_total_days_ls.append(delay_total_days)
        delay_days_mean_ls.append(delay_days_mean)
        total_payment_ls.append(total_payment)
        total_pay_owe_diff_ls.append(total_pay_owe_diff)
        total_pay_owe_diff_ratio_ls.append(total_pay_owe_diff_ratio)
    
    
    curr = [i[1] for i in prev_curr_map]
    
    inst_agg_df["sk_id_prev"] = group_keys[:]
    inst_agg_df["sk_id_curr"] = curr[:]
    inst_agg_df["total_inst_count"] = total_inst_count_ls
    inst_agg_df["fully_paid_ratio"] = fully_paid_ratio_ls
    inst_agg_df["delay_inst_ratio"] = delay_inst_ratio_ls
    inst_agg_df["delay_total_days"] = delay_total_days_ls
    inst_agg_df["total_payment"] = total_payment_ls
    inst_agg_df["total_pay_owe_diff"] = total_pay_owe_diff_ls
    inst_agg_df["total_pay_owe_diff_ratio"] = total_pay_owe_diff_ratio_ls
    
    joblib.dump(inst_agg_df, "data/inst_agg_df")

##### credit_card_agg_df

In [ ]:
# cc_agg_df
def cc_agg_df():
    """
    Aggregate credit card df data
    """
    cc_status_map = {
        "Completed": 1,
        "Active": 2,
        "Signed": 3,
        "Approved": 4,
        "Sent proposal": 5,
        "Demand": 6,
        "Refused": 7,
    }

    cc_df["contract_stat"] = cc_df.name_contract_status.apply(lambda i: cc_status_map[i])

    prev_curr = cc_df[
        [
            "sk_id_prev",
            "sk_id_curr",
        ]
    ].groupby(["sk_id_prev", "sk_id_curr"])

    prev_curr_map = list(prev_curr.groups.keys())

    cc_group = cc_df[
        [
            "sk_id_prev",
            "sk_id_curr",
            "months_balance",
            "amt_credit_limit_actual",
            "amt_drawings_current",
            "amt_drawings_atm_current",
            "amt_drawings_pos_current",
            "amt_inst_min_regularity",
            "amt_payment_total_current",
            "amt_recivable",  # org and fin
            "amt_total_receivable",
            "amt_receivable_principal",
            "contract_stat",
            "sk_dpd",
            "sk_dpd_def",
        ]
    ].groupby(["sk_id_prev"])

    group_keys = list(cc_group.groups.keys())


    total_months_ls = []
    atm_draw_ratio_avg_ls = []
    pos_draw_ratio_avg_ls = []
    total_monthly_draw_avg_ls = []
    credit_lim_fin_org_ratio_ls = []

    total_still_owes_ls = []
    total_still_owes_int_princ_ratio_ls = []
    total_paid_ls = []
    total_interest_paid_ls = []
    interest_rate_ls = []
    pay_owe_ratio_total_ls = []
    pay_less_than_min_reg_avg_ls = []

    sk_dpd_sum_ls = []
    sk_dpd_mean_ls = []
    sk_dpd_def_sum_ls = []
    sk_dpd_def_mean_ls = []
    contract_stat_ls = []


    for i in group_keys[:]:
        gp = cc_group.get_group((i,)).sort_values(by="months_balance")
        total_months = len(gp.months_balance)
        # adjustedddd
        atm_draw_ratio_avg = (
            (gp.amt_drawings_atm_current.sum() / gp.amt_drawings_current.sum())
            if gp.amt_drawings_current.sum() != 0
            else 0
        )
        pos_draw_ratio_avg = (
            (gp.amt_drawings_pos_current.sum() / gp.amt_drawings_current.sum())
            if gp.amt_drawings_current.sum() != 0
            else 0
        )

        total_monthly_draw_avg = (
            (gp.amt_drawings_current.sum() / total_months) if total_months != 0 else 0
        )

        credit_lim_fin_org_ratio = (
            (gp.amt_credit_limit_actual.iloc[-1] / gp.amt_credit_limit_actual.iloc[0])
            if gp.amt_credit_limit_actual.iloc[0] != 0
            else 0
        )

        total_still_owes = gp.amt_total_receivable.iloc[-1]

        total_still_owes_int_princ_ratio = (
            (
                (gp.amt_recivable.iloc[-1] - gp.amt_receivable_principal.iloc[-1])
                / gp.amt_recivable.iloc[-1]
            )
            if gp.amt_recivable.iloc[-1] != 0
            else 0
        )

        total_paid = gp.amt_payment_total_current.sum()

        total_interest_paid = (
            gp.amt_payment_total_current.sum() - gp.amt_drawings_current.sum()
        )
        interest_rate = (
            (total_interest_paid / gp.amt_drawings_current.sum())
            if gp.amt_drawings_current.sum() != 0
            else 0
        )

        pay_owe_ratio_total = (
            (gp.amt_drawings_current.sum() / gp.amt_payment_total_current.sum())
            if gp.amt_payment_total_current.sum() != 0
            else 0
        )

        pay_less_than_min_reg_avg = (
            (sum(gp.amt_inst_min_regularity > gp.amt_payment_total_current) / total_months)
            if total_months != 0
            else 0
        )

        sk_dpd_sum = gp.sk_dpd.sum()
        sk_dpd_mean = sk_dpd_sum / len(gp.sk_dpd)
        sk_dpd_def_sum = gp.sk_dpd_def.sum()
        sk_dpd_def_mean = sk_dpd_def_sum / len(gp.sk_dpd_def)
        contract_stat = gp.contract_stat.mean()

        total_months_ls.append(total_months)
        atm_draw_ratio_avg_ls.append(atm_draw_ratio_avg)
        pos_draw_ratio_avg_ls.append(pos_draw_ratio_avg)
        total_monthly_draw_avg_ls.append(total_monthly_draw_avg)
        credit_lim_fin_org_ratio_ls.append(credit_lim_fin_org_ratio)
        total_still_owes_ls.append(total_still_owes)

        total_still_owes_int_princ_ratio_ls.append(total_still_owes_int_princ_ratio)
        total_paid_ls.append(total_paid)
        total_interest_paid_ls.append(total_interest_paid)
        interest_rate_ls.append(interest_rate)

        pay_owe_ratio_total_ls.append(pay_owe_ratio_total)
        pay_less_than_min_reg_avg_ls.append(pay_less_than_min_reg_avg)
        sk_dpd_sum_ls.append(sk_dpd_sum)
        sk_dpd_mean_ls.append(sk_dpd_mean)
        sk_dpd_def_sum_ls.append(sk_dpd_def_sum)
        sk_dpd_def_mean_ls.append(sk_dpd_def_mean)
        contract_stat_ls.append(contract_stat)


    cc_agg_df = pd.DataFrame()


    curr = [i[1] for i in prev_curr_map]

    cc_agg_df["sk_id_prev"] = group_keys[:]
    cc_agg_df["sk_id_curr"] = curr[:]
    cc_agg_df["total_months"] = total_months_ls

    cc_agg_df["atm_draw_ratio_avg"] = atm_draw_ratio_avg_ls
    cc_agg_df["pos_draw_ratio_avg"] = pos_draw_ratio_avg_ls
    cc_agg_df["total_monthly_draw_avg"] = total_monthly_draw_avg_ls
    cc_agg_df["credit_lim_fin_org_ratio"] = credit_lim_fin_org_ratio_ls
    cc_agg_df["total_still_owes"] = total_still_owes_ls
    cc_agg_df["total_still_owes_int_princ_ratio"] = total_still_owes_int_princ_ratio_ls
    cc_agg_df["total_paid"] = total_paid_ls

    cc_agg_df["total_interest_paid"] = total_interest_paid_ls
    cc_agg_df["interest_rate"] = interest_rate_ls
    cc_agg_df["pay_owe_ratio_total"] = pay_owe_ratio_total_ls
    cc_agg_df["pay_less_than_min_reg_avg"] = pay_less_than_min_reg_avg_ls

    cc_agg_df["sk_dpd_sum"] = sk_dpd_sum_ls
    cc_agg_df["sk_dpd_mean"] = sk_dpd_mean_ls
    cc_agg_df["sk_dpd_def_sum"] = sk_dpd_def_sum_ls
    cc_agg_df["sk_dpd_def_mean"] = sk_dpd_def_mean_ls
    cc_agg_df["contract_stat"] = contract_stat_ls

    cc_agg_df["card_auto_pay_online"] = np.where(
        cc_agg_df.sk_id_prev.isin(card_auto_pay_online_id), 1, 0
    )

    assert cc_agg_df.shape[0] == 87452, "cc_agg_df shape not match"


    joblib.dump(cc_agg_df, "data/cc_agg_df")

##### pos_cash_agg_df

In [ ]:
# pos_agg_df 
def pos_agg_df():
    """
    Aggregate pod_cash df data
    """

    pos_agg_df = pd.DataFrame()
    
    prev_curr = pos_df[
        [
            "sk_id_prev",
            "sk_id_curr",
        ]
    ].groupby(["sk_id_prev", "sk_id_curr"])
    
    prev_curr_map = list(prev_curr.groups.keys())
    
    pos_group = pos_df[
        [
            "sk_id_prev",
            "sk_id_curr",
            "months_balance",
            "cnt_instalment",
            "sk_dpd",
            "sk_dpd_def",
        ]
    ].groupby(["sk_id_prev"])
    
    group_keys = list(pos_group.groups.keys())
    
    sk_dpd_sum_ls = []
    sk_dpd_mean_ls = []
    sk_dpd_def_sum_ls = []
    sk_dpd_def_mean_ls = []
    cnt_inst_org_ls = []
    cnt_inst_fin_ls = []
    total_months_ls = []
    
    for i in group_keys:
        gp = pos_group.get_group((i,)).sort_values(by="months_balance")
        sk_dpd_sum = gp.sk_dpd.sum()
        sk_dpd_mean = sk_dpd_sum / len(gp.sk_dpd)
        sk_dpd_def_sum = gp.sk_dpd_def.sum()
        sk_dpd_def_mean = sk_dpd_def_sum / len(gp.sk_dpd_def)
        cnt_inst_org = gp.cnt_instalment.tolist()[0]
        cnt_inst_fin = gp.cnt_instalment.tolist()[-1]
        total_months = len(gp.months_balance)
    
        sk_dpd_sum_ls.append(sk_dpd_sum),
        sk_dpd_mean_ls.append(sk_dpd_mean),
        sk_dpd_def_sum_ls.append(sk_dpd_def_sum),
        sk_dpd_def_mean_ls.append(sk_dpd_def_mean),
        cnt_inst_org_ls.append(cnt_inst_org),
        cnt_inst_fin_ls.append(cnt_inst_fin),
        total_months_ls.append(total_months)
    
    curr = [i[1] for i in prev_curr_map]
    
    pos_agg_df["sk_id_prev"] = group_keys
    pos_agg_df["sk_id_curr"] = curr
    pos_agg_df["sk_dpd_sum"] = sk_dpd_sum_ls
    pos_agg_df["sk_dpd_mean"] = sk_dpd_mean_ls
    pos_agg_df["sk_dpd_def_sum"] = sk_dpd_def_sum_ls
    pos_agg_df["sk_dpd_def_mean"] = sk_dpd_def_mean_ls
    pos_agg_df["cnt_inst_org"] = cnt_inst_org_ls
    pos_agg_df["cnt_inst_fin"] = cnt_inst_fin_ls
    pos_agg_df["total_months"] = total_months_ls
    
    
    status_map = {
        "Completed": 1,
        "Active": 2,
        "Amortized debt": 3,
        "Signed": 4,
        "Approved": 5,
        "Returned to the store": 6,
        "Canceled": 7,
        "Demand": 8,
        "XNA": np.nan,
    }
    
    name_contract_ord = pos_df.name_contract_status.apply(lambda i: status_map[i])
    pos_df["name_contract_ord"] = name_contract_ord
    pos_agg_df["name_contract_ord"] = pos_df.groupby("sk_id_prev").name_contract_ord.mean().tolist()
    
    joblib.dump(pos_agg_df, "data/pos_agg_df")
    

##### bu_bal_agg_df

In [ ]:
status_map = {"C": 0, "X": np.nan, "0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5}

gp = bu_bal_df.groupby(["sk_id_bureau"])[["months_balance"]]
idx = gp.idxmax()
gp_keys = gp.groups.keys()
bu_bal_agg_df = pd.DataFrame()
bu_bal_agg_df["sk_id_bureau"] = gp_keys
bu_bal_df["status"] = bu_bal_df.status.apply(lambda i: status_map[i])
status_mean = bu_bal_df.groupby("sk_id_bureau").status.mean()
bu_bal_agg_df = bu_bal_agg_df.set_index("sk_id_bureau")
bu_bal_agg_df["status"] = status_mean
bu_bal_agg_df = bu_bal_agg_df.reset_index()

joblib.dump(bu_bal_agg_df, "data/bu_bal_agg_df")

##### bu_agg_df

In [ ]:
# bu_agg_df
def bu_agg_df():
    """
    Aggregate bureau df data
    """

    bu_agg_df = pd.DataFrame()
    bu_group = bu_mg_df[
        [
            "sk_id_bureau",
            "sk_id_curr",
            "credit_active",
            "credit_currency",
            "days_credit",
            "credit_day_overdue",
            "amt_credit_max_overdue",
            "cnt_credit_prolong",
            "amt_credit_sum",
            "amt_credit_sum_debt",
            "amt_annuity",
            "status",
        ]
    ].groupby(["sk_id_curr"])
    
    group_keys = list(bu_group.groups.keys())
    
    total_credit_count_ls = []
    active_credit_count_ls = []
    currency_1_ratio_ls = []
    days_credit_active_mean_ls = []
    credit_day_overdue_mean_ls = []
    credit_day_overdue_count_ls = []
    amt_credit_max_overdue_mean_ls = []
    cnt_credit_prolong_mean_ls = []
    amt_credit_sum_mean_ls = []
    amt_credit_sum_debt_mean_ls = []
    amt_annuity_mean_ls = []
    status_mean_ls = []
    
    for i in group_keys[:]:
        gp = bu_group.get_group((i,))
    
        total_credit_count = gp.credit_active.count()
        active_credit_count = gp.credit_active.sum()
        currency_1_ratio = (gp.credit_currency == 1).sum() / \
            gp.credit_currency.count()
        days_credit_active_mean = gp.days_credit.mean()
        credit_day_overdue_mean = gp.credit_day_overdue.mean()
        credit_day_overdue_count = gp.credit_day_overdue.count()
        amt_credit_max_overdue_mean = gp.amt_credit_max_overdue.mean()
        cnt_credit_prolong_mean = gp.cnt_credit_prolong.mean()
        amt_credit_sum_mean = gp.amt_credit_sum.mean()
        amt_credit_sum_debt_mean = gp.amt_credit_sum_debt.mean()
        amt_annuity_mean = gp.amt_annuity.mean()
        status_mean = gp.status.mean()
    
        total_credit_count_ls.append(total_credit_count)
        active_credit_count_ls.append(active_credit_count)
        currency_1_ratio_ls.append(currency_1_ratio)
        days_credit_active_mean_ls.append(days_credit_active_mean)
        credit_day_overdue_mean_ls.append(credit_day_overdue_mean)
        credit_day_overdue_count_ls.append(credit_day_overdue_count)
        amt_credit_max_overdue_mean_ls.append(amt_credit_max_overdue_mean)
        cnt_credit_prolong_mean_ls.append(cnt_credit_prolong_mean)
        amt_credit_sum_mean_ls.append(amt_credit_sum_mean)
        amt_credit_sum_debt_mean_ls.append(amt_credit_sum_debt_mean)
        amt_annuity_mean_ls.append(amt_annuity_mean)
        status_mean_ls.append(status_mean)
    
    curr = [i[1] for i in bu_curr_map]
    
    bu_agg_df["sk_id_curr"] = group_keys[:]
    bu_agg_df["total_credit_count"] = total_credit_count_ls
    bu_agg_df["active_credit_count"] = active_credit_count_ls
    bu_agg_df["currency_1_ratio"] = currency_1_ratio_ls
    bu_agg_df["days_credit_active_mean"] = days_credit_active_mean_ls
    bu_agg_df["credit_day_overdue_mean"] = credit_day_overdue_mean_ls
    bu_agg_df["credit_day_overdue_count"] = credit_day_overdue_count_ls
    bu_agg_df["amt_credit_max_overdue_mean"] = amt_credit_max_overdue_mean_ls
    bu_agg_df["cnt_credit_prolong_mean"] = cnt_credit_prolong_mean_ls
    bu_agg_df["amt_credit_sum_mean"] = amt_credit_sum_mean_ls
    bu_agg_df["amt_credit_sum_debt_mean"] = amt_credit_sum_debt_mean_ls
    bu_agg_df["amt_annuity_mean"] = amt_annuity_mean_ls
    bu_agg_df["status_mean"] = status_mean_ls
    
    
    joblib.dump(bu_agg_df, "data/bu_agg_df")

##### comb_agg_df

In [ ]:
def comb_agg_df():
    """
    Aggregate comb_df data
    """

    comb_df = joblib.load('data/comb_df')
    comb_group = comb_df.groupby(["sk_id_curr", "name_contract_type"])

    group_keys = list(comb_group.groups.keys())
    len(group_keys)

    num_cols = comb_df.select_dtypes(["float", "int"]).columns

    # init comb_agg_df
    comb_agg_df = [[0]*62]
    for i in group_keys:
        curr_id = i[0]
        gp = comb_group.get_group(i).iloc[:, 2:]
        temp = pd.DataFrame()
        for i in gp:
            temp["sk_id_curr"] = curr_id
            if i in num_cols:
                temp[i] = np.mean(gp[i])
            else:
                temp[i] = gp[i].mode()

        comb_agg_df = np.vstack([comb_agg_df, temp])

    comb_cols = comb_df.iloc[:, 1:].columns.tolist()
    final = pd.DataFrame(comb_agg_df.iloc[1:, :], columns=comb_cols)

##### comb_agg_pivot_df

In [ ]:
def comb_agg_pivot_df():
    """
    Pivot comb_agg_df on loan types
    """

    cols = comb_agg_df.iloc[:, 2:].columns.tolist()
    
    pivot_t = comb_agg_df.pivot(
        index="sk_id_curr",
        columns="name_contract_type",
        values=cols,
    ).reset_index()
    
    
    pv_cols = pivot_t.columns.tolist()
    
    new_cols = [i[0] if i[0] == "sk_id_curr" else i[1] + "_" + i[0]
                for i in pv_cols]
    
    pivot_t_new = pd.DataFrame(pivot_t.values, columns=new_cols)
    
    joblib.dump(pivot_t_new, 'data/comb_agg_pivot_df')